In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PromptTemplate,
    FewShotChatMessagePromptTemplate
)
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=00
)

## Practice 2-1

In [3]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser

output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()

prompt = PromptTemplate(
    template="List five {subject}. \n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

chain = prompt | model | output_parser

result = chain.invoke({"subject": "음식"})

print(result)

['피자', '햄버거', '샌드위치', '라면', '김치찌개']


## Practice 2-2

In [7]:
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

class Sentiment(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NEUTRAL = "보통"
    
parser = EnumOutputParser(enum=Sentiment)
format_instructions = parser.get_format_instructions()

template = """
당신은 텍스트 감정 분석 전문가입니다.
다음 텍스트의 감정을 분석하고, 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "중립" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

fixing_parser = OutputFixingParser.from_llm(parser=parser, llm=model)

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요."
]

def safe_sentiment_analysis(text, use_fixing_parser=True):
    """안전한 감정 분석 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | (fixing_parser if use_fixing_parser else parser)
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."
    
def run_sentiment_analysis():
    
    success_count = 0
    total_count = len(texts)
    
    for i, text in enumerate(texts, 1):
        print(f"\n{i}. 텍스트: {text}")
        
        # OutputFixingParser 사용
        result, error = safe_sentiment_analysis(text, use_fixing_parser=True)
        
        if result:
            print(f"   감정: {result.value} ")
            success_count += 1
        else:
            print(f"   오류: {error} ")
            
            # 기본 파서로 재시도
            print("   기본 파서로 재시도...")
            result2, error2 = safe_sentiment_analysis(text, use_fixing_parser=False)
            
            if result2:
                print(f"   감정: {result2.value} (기본 파서 성공)")
                success_count += 1
            else:
                print(f"   재시도 실패: {error2} ")
    
try:
    run_sentiment_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")


1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 


## Practice 2-3

In [9]:
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List

class StudentInfo(BaseModel):
    name : str = Field(description="학생의 이름")
    age: int = Field(description="학생의 나이")
    major: str = Field(description="학생의 전공")
    hobbies: List[str] = Field(description="학생의 취미")
    goal: str = Field(description="학생의 목표")
    
parser = PydanticOutputParser(pydantic_object=StudentInfo)
template = """
학생의 자기소개를 보고 학생의 정보를 추출해주세요.
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=parser.get_format_instructions())

query = "안녕하세요! 저는 김민수이고 22살입니다. 컴퓨터공학을 전공하고 있어요. 취미로는 게임하기, 영화보기, 코딩을 좋아합니다. 앞으로 훌륭한 개발자가 되는 것이 목표입니다."

chain = prompt | model | parser
result = chain.invoke({"query": query})

print(f"이름: {result.name}")
print(f"나이: {result.age}")
print(f"전공: {result.major}")
print(f"취미: {', '.join(result.hobbies)}")
print(f"목표: {result.goal}")

이름: 김민수
나이: 22
전공: 컴퓨터공학
취미: 게임하기, 영화보기, 코딩
목표: 훌륭한 개발자가 되는 것


## Practice 2-4

In [10]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from pprint import pprint

response_scemas = [
    ResponseSchema(name="destination", description="여행지/목적지 (도시명이나 지역명)"),
    ResponseSchema(name="duration", description="여행 기간 (예: 2박 3일, 1주일, 3일 등)"),
    ResponseSchema(name="budget", description="여행 예산이나 비용 (예: 30만원, 50만원 등)"),
    ResponseSchema(name="rating", description="여행 만족도나 추천도 (1-5점 사이의 숫자만)"),
    ResponseSchema(name="activities", description="주요 활동들의 리스트 (배열 형태로 각 활동을 나열)")
]

parser = StructuredOutputParser.from_response_schemas(response_scemas)
format_instructions = parser.get_format_instructions()

prompt = PromptTemplate(
    template="""
    당신은 여행 전문가입니다. 여행 후기나 계획을 분석하고 여행지, 기간, 예산 추천도(1~5점), 주요 활동리스트를 구조화된 형태로 추출하세요.
    
    {format_instructions}
    여행 후기: "{review}"
    """,
    input_variables=["review"],
    partial_variables={"format_instructions": format_instructions}
)

chain = prompt | model | parser
result = chain.invoke({
    "review": "지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
})

pprint(result)

{'activities': ['해운대 바다구경', '자갈치시장에서 회 먹기', '감천문화마을 구경'],
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박3일',
 'rating': '4'}
